In [1]:
import sys
sys.path.insert(0, "../..")

In [2]:
import numpy as np
import scipy.io
from gudhi.representations.preprocessing import BirthPersistenceTransform, DiagramScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.mixture import GaussianMixture
from sklearn.pipeline import make_pipeline
from preprocessing import *
from persistent_bow import *
from visualization import *

In [3]:
data_path = "./pd.mat"
data_mat = scipy.io.loadmat(data_path)
data = data_mat["pds"]

In [4]:
# y - labels - type of diagram [0-7]
# We transpose to iterate over diagram list
y = []
for label, diagram_list in enumerate(data.T):
    y += len(diagram_list) * [label]
y = np.array(y)

# data.T = 2d array of diagrams as row, column as type
X = data.T.reshape(-1)

pipe = make_pipeline(BirthPersistenceTransform(), 
               DiagramScaler(use=True, 
                             scalers=[
                                 ((0,), MaxAbsScaler(copy=False)),
                                 ((1,), MaxAbsScaler(copy=False))]))

preprocessed = np.array(pipe.fit_transform(X, y))

In [5]:
vectorizer = StablePersistentBow(GaussianMixture(10, random_state=42), transformator=None, scaler=None)
vectorizer.fit(preprocessed)

StablePersistentBow(mixture=GaussianMixture(covariance_type='full',
                                            init_params='kmeans', max_iter=100,
                                            means_init=None, n_components=10,
                                            n_init=1, precisions_init=None,
                                            random_state=42, reg_covar=1e-06,
                                            tol=0.001, verbose=0,
                                            verbose_interval=10,
                                            warm_start=False,
                                            weights_init=None),
                    sampler=None, scaler=None, transformator=None)

In [21]:
vectorizer.transform(preprocessed)

array([[2.53487279e+00, 1.86202252e-15, 5.00040955e+00, ...,
        3.33183313e-02, 2.57447159e-04, 1.92045135e+00],
       [2.15864635e+00, 3.77881259e-17, 3.94513696e+00, ...,
        3.72132963e-02, 1.77836965e-04, 3.84170966e+00],
       [2.87913426e+00, 4.26817386e-18, 5.20539368e+00, ...,
        3.54263013e-02, 3.41718916e-04, 2.55879331e+00],
       ...,
       [1.39301648e+00, 1.88743538e-05, 8.28256904e-01, ...,
        2.31313302e-02, 7.72643546e-03, 8.33898704e-01],
       [1.32994773e+00, 2.81233576e-09, 1.03973227e+00, ...,
        8.93675597e-03, 4.09588368e-03, 8.94315541e-01],
       [1.68958479e+00, 1.65908079e-10, 1.85591358e+00, ...,
        6.48391659e-03, 4.64488164e-03, 8.70835916e-01]])

In [55]:
x = np.linspace(0, 1)
y = np.linspace(0, 1)
X, Y = np.meshgrid(x, y)
XX = np.array([X.ravel(), Y.ravel()]).T
Z = -vectorizer.mixture.score_samples(XX)

In [22]:
plt.contour(X, Y, Z.reshape(X.shape))

NameError: name 'Y' is not defined